## Import Libraries

In [1]:
# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np
#pd.set_option('display.max_columns', None)  
import seaborn as sns
#pd.set_option('display.max_rows', None)
#from tqdm._tqdm_notebook import tqdm_notebook as tqdm
#from tqdm import tqdm

## Import two_sides dataset - drug-drug interaction dataset

In [2]:
#Import two sides dataset. TWOSIDES - data of side effects of pairs of drugs

two_sides_full=pd.read_csv('data/TWOSIDES.csv.xz.csv', compression='xz', \
                      header=0, sep=',', quotechar='"')


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,4,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
two_sides_full.head(2)

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
0,10355,Temazepam,136411,sildenafil,10003239,Arthralgia,7,149,24,1536,2.91667,0.421275,0.0448718
1,1808,Bumetanide,7824,Oxytocin,10003239,Arthralgia,1,13,2,138,5.0,1.19224,0.0714286


In [4]:
two_sides_full.query('(drug_1_concept_name=="Tamoxifen") & \
(drug_2_concept_name=="sildenafil") &(condition_concept_name=="Diarrhoea")')

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency


In [5]:
two_sides_full.query('(drug_1_concept_name=="Acetaminophen") & \
(drug_2_concept_name=="glycopyrronium") &(condition_concept_name=="Nausea")')

,drug_1_rxnorn_id,drug_1_concept_name,drug_2_rxnorm_id,drug_2_concept_name,condition_meddra_id,condition_concept_name,A,B,C,D,PRR,PRR_error,mean_reporting_frequency
79,161,Acetaminophen,1546438,glycopyrronium,10028813,Nausea,3,20,6,224,5.0,0.672439,0.130435
1549,161,Acetaminophen,1546438,glycopyrronium,10028813,Nausea,3,20,6,224,5.0,0.672439,0.130435


In [6]:
#Drop the unwanted columns 
two_sides=two_sides_full.drop(['A', 'B', 'C', 'D', 'PRR', 'PRR_error', \
                          'mean_reporting_frequency', 'drug_1_rxnorn_id', \
                         'drug_2_rxnorm_id', 'condition_meddra_id'], axis=1)

In [7]:
#rename columns
two_sides.columns=['drug_1', 'drug_2', 'condn_name']

In [8]:
two_sides.head()

,drug_1,drug_2,condn_name
0,Temazepam,sildenafil,Arthralgia
1,Bumetanide,Oxytocin,Arthralgia
2,POLYETHYLENE GLYCOL 3350,Hydroxychloroquine,Arthralgia
3,Tamoxifen,Prednisone,Diarrhoea
4,Temazepam,sildenafil,Diarrhoea


In [9]:
#There are duplicates for certain drug pairs: remove duplicates
two_sides=two_sides.drop_duplicates(subset=['drug_1', 'drug_2', 'condn_name'])

In [10]:
two_sides.query('(drug_1=="Temazepam") & \
(drug_2=="sildenafil") &(condn_name=="Diarrhoea")')

,drug_1,drug_2,condn_name
4,Temazepam,sildenafil,Diarrhoea


In [11]:
two_sides_r=two_sides.query('condn_name=="Nausea" | condn_name =="Dyspnoea" | \
condn_name =="Diarrhoea" | condn_name =="Vomiting" | condn_name =="Pyrexia" | \
condn_name =="Pneumonia"')

In [12]:
two_sides_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 926781 entries, 3 to 42920304
Data columns (total 3 columns):
drug_1        926781 non-null object
drug_2        926781 non-null object
condn_name    926781 non-null object
dtypes: object(3)
memory usage: 28.3+ MB


In [13]:
#dummize the columns
#pd.get_dummies(df, columns=['key2']).groupby(['key1'], as_index=False).sum()
#pd.concat([df, pd.get_dummies(df.ANNOT)], 1).groupby(['CHR','BP','SNP','CM']).sum()
two_sides_r1=pd.concat([two_sides_r, pd.get_dummies(two_sides_r.condn_name)], 1)\
.groupby(['drug_1', 'drug_2']).sum().reset_index()
#two_sides_r1=pd.get_dummies(two_sides_r, columns=['condn_name'])

In [14]:
#convert the drug names to lower case
two_sides_r1['drug_1']=two_sides_r1.drug_1.str.lower()
two_sides_r1['drug_2']=two_sides_r1.drug_2.str.lower()

In [15]:
two_sides_r1.head(2)

,drug_1,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting
0,6-aminocaproic acid,acetaminophen,1,1,1,1,1,1
1,6-aminocaproic acid,acyclovir,0,1,1,1,1,0


In [16]:
two_sides_r1.nunique()

drug_1       1677
drug_2       1805
Diarrhoea       2
Dyspnoea        2
Nausea          2
Pneumonia       2
Pyrexia         2
Vomiting        2
dtype: int64

In [17]:
two_sides_r1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204571 entries, 0 to 204570
Data columns (total 8 columns):
drug_1       204571 non-null object
drug_2       204571 non-null object
Diarrhoea    204571 non-null uint8
Dyspnoea     204571 non-null uint8
Nausea       204571 non-null uint8
Pneumonia    204571 non-null uint8
Pyrexia      204571 non-null uint8
Vomiting     204571 non-null uint8
dtypes: object(2), uint8(6)
memory usage: 4.3+ MB


In [18]:
#two_sides_r1.query('(drug_1=="Temazepam") & (drug_2=="sildenafil")')

In [19]:
#two_sides_r1.query('Diarrhoea==2')

In [20]:
#create a list of drugs from drug_1 column
two_sides_drug1list=two_sides_r1.drug_1.tolist()
type(two_sides_drug1list)

list

In [21]:
#check an element in a list
#'bupropion' in two_sides_drug1list

In [22]:
#create a list of drugs from drug_2 column
two_sides_drug2list=two_sides_r1.drug_2.tolist()

In [23]:
#Import chemical substr dataset from pawel's dataset
chem_str=pd.read_csv('data/chemical_substr.txt', sep='\t')

In [24]:
chem_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Columns: 882 entries, Unnamed: 0 to SUB881
dtypes: int64(881), object(1)
memory usage: 6.0+ MB


In [25]:
chem_str.head(2)

,Unnamed: 0,SUB1,SUB2,SUB3,SUB4,SUB5,SUB6,SUB7,SUB8,SUB9,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,carnitine,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,GABA,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#rename column 1
chem_str=chem_str.rename(columns={"Unnamed: 0": "drug_name"})

In [27]:
chem_str['drug_name']=chem_str.drug_name.str.lower()

In [28]:
chem_str.iloc[:,1:889]=chem_str.iloc[:,1:889].apply(pd.to_numeric, downcast='integer')

In [29]:
chem_str.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Columns: 882 entries, drug_name to SUB881
dtypes: int8(881), object(1)
memory usage: 771.1+ KB


# Merges
##Merge1- merge drug_1 from two_sides to chem_str drug_name and get the chem substr
##merge 2 - merge drug_2 from two_sides to chem_str drug_name to get their chem substr
Step 3- create list of each of the chem substr from merge 1 and merge 2 separately
step 4 - find maximum of the binary profiles
step 5 - Merge the file from step 4 back to combination of merge 1 and 2

In [70]:
#Merge1
tscs_merge1=pd.merge(two_sides_r1, chem_str, left_on='drug_1', right_on='drug_name', \
                     how='inner')

In [71]:
#tscs_merge1=tscs_merge1.drop(['drug_2'], axis=1)
#.drop(['drug_name']), axis=1

In [72]:
tscs_merge1.head(2)

,drug_1,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,drug_name,SUB1,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,acarbose,acebutolol,0,1,0,0,1,0,acarbose,1,...,0,0,0,0,0,0,0,0,0,0
1,acarbose,acenocoumarol,1,1,0,0,1,1,acarbose,1,...,0,0,0,0,0,0,0,0,0,0


In [33]:
tscs_merge1.iloc[123069:123073, ]

,drug_1,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,drug_name,SUB1,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
123069,valacyclovir,voriconazole,1,1,1,1,1,1,valacyclovir,1,...,0,0,0,0,0,0,0,0,0,0
123070,valacyclovir,ziprasidone,0,0,1,0,1,1,valacyclovir,1,...,0,0,0,0,0,0,0,0,0,0
123071,valacyclovir,zoledronic acid,1,1,1,1,1,1,valacyclovir,1,...,0,0,0,0,0,0,0,0,0,0
123072,valacyclovir,zolmitriptan,0,1,1,0,1,1,valacyclovir,1,...,0,0,0,0,0,0,0,0,0,0


In [34]:
tscs_merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126951 entries, 0 to 126950
Columns: 890 entries, drug_1 to SUB881
dtypes: int8(881), object(3), uint8(6)
memory usage: 111.3+ MB


In [73]:
tscs_merge1_list1=tscs_merge1.iloc[:, 9:891]
tscs_merge1_list=tscs_merge1_list1.values.tolist()

#tscs_merge1_list=tcsc.reset_index()[['index', 'Names', 'Births']].values.astype(str).tolist()

tscs_merge1_list=pd.Series(tscs_merge1_list)

In [74]:
tscs_merge1_list1.head(2)

,SUB1,SUB2,SUB3,SUB4,SUB5,SUB6,SUB7,SUB8,SUB9,SUB10,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [63]:
tscs_merge1_list

0         [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
1         [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
2         [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
3         [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
4         [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
                                ...                        
126946    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
126947    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
126948    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
126949    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
126950    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
Length: 126951, dtype: object

In [37]:
#Merge2
tscs_merge2=pd.merge(two_sides_r1, chem_str, left_on='drug_2', right_on='drug_name', \
                     how='inner').drop(['drug_name'], axis=1)

In [38]:
tscs_merge2=tscs_merge2.drop(['drug_1'], axis=1)

In [39]:
tscs_merge2.head(2)

,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,SUB1,SUB2,SUB3,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,acyclovir,0,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,acyclovir,1,1,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
tscs_merge2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123069 entries, 0 to 123068
Columns: 888 entries, drug_2 to SUB881
dtypes: int8(881), object(1), uint8(6)
memory usage: 106.0+ MB


In [41]:
tscs_merge2.iloc[123069:, ]

,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,SUB1,SUB2,SUB3,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881


In [58]:
tscs_merge2_list2=tscs_merge2.iloc[:, 7:889]
tscs_merge2_list=tscs_merge2_list2.values.tolist()
tscs_merge2_list=pd.Series(tscs_merge2_list)

In [59]:
tscs_merge2_list2.head(2)

,SUB1,SUB2,SUB3,SUB4,SUB5,SUB6,SUB7,SUB8,SUB9,SUB10,...,SUB872,SUB873,SUB874,SUB875,SUB876,SUB877,SUB878,SUB879,SUB880,SUB881
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [60]:
tscs_merge2_list

0         [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...
1         [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...
2         [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...
3         [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...
4         [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...
                                ...                        
123064    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
123065    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
123066    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...
123067    [1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...
123068    [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...
Length: 123069, dtype: object

In [85]:
len(tscs_merge2_list[123068])

881

In [61]:
type(tscs_merge2_list)

pandas.core.series.Series

### step 4 - find maximum of the binary profiles

In [98]:
chem_str_combind=np.maximum(tscs_merge1_list[0:123069], tscs_merge2_list[0:123069])

In [89]:
len(chem_str_combind[0])

881

In [99]:
type(chem_str_combind)

pandas.core.series.Series

In [100]:
#convert the ADE_str list to dataframe 
chem_str_combind=pd.DataFrame(chem_str_combind.tolist())

In [101]:
chem_str_combind.head(2)

,0,1,2,3,4,5,6,7,8,9,...,871,872,873,874,875,876,877,878,879,880
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [102]:
#add suffix to column names
chem_str_combind=chem_str_combind.add_suffix('_sub')

#convert to int8
chem_str_combind=chem_str_combind.apply(pd.to_numeric, downcast='integer')
#type(se_binary_ade)

In [103]:
chem_str_combind.head(2)

,0_sub,1_sub,2_sub,3_sub,4_sub,5_sub,6_sub,7_sub,8_sub,9_sub,...,871_sub,872_sub,873_sub,874_sub,875_sub,876_sub,877_sub,878_sub,879_sub,880_sub
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### Merge drug fields with max chem_str binary profile(chem_str_combind)

In [104]:
tscs_merge1_drug_se=tscs_merge1.iloc[:123069, :9]
tscs_merge1_drug_se.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123069 entries, 0 to 123068
Data columns (total 9 columns):
drug_1       123069 non-null object
drug_2       123069 non-null object
Diarrhoea    123069 non-null uint8
Dyspnoea     123069 non-null uint8
Nausea       123069 non-null uint8
Pneumonia    123069 non-null uint8
Pyrexia      123069 non-null uint8
Vomiting     123069 non-null uint8
drug_name    123069 non-null object
dtypes: object(3), uint8(6)
memory usage: 4.5+ MB


In [105]:
tscs_merge1_drug_se.head(2)

,drug_1,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,drug_name
0,acarbose,acebutolol,0,1,0,0,1,0,acarbose
1,acarbose,acenocoumarol,1,1,0,0,1,1,acarbose


In [106]:
final_model_data=pd.concat([tscs_merge1_drug_se, chem_str_combind], axis=1)

In [111]:
final_model_data.tail()

,drug_1,drug_2,Diarrhoea,Dyspnoea,Nausea,Pneumonia,Pyrexia,Vomiting,0_sub,1_sub,...,871_sub,872_sub,873_sub,874_sub,875_sub,876_sub,877_sub,878_sub,879_sub,880_sub
123064,valacyclovir,vardenafil,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
123065,valacyclovir,varenicline,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
123066,valacyclovir,venlafaxine,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
123067,valacyclovir,vinorelbine,0,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
123068,valacyclovir,vitamin d3,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [110]:
final_model_data=final_model_data.drop(['drug_name'],axis=1)

In [112]:
#write the final model data to csv
final_model_data.to_csv('data/final_model_data', index=False)

In [55]:
#convert series to dataframe

In [56]:
#chem_drug1_twosides=chem_str[chem_str['drug_name'].isin(two_sides_drug1list)]

#chem_drug2_twosides=chem_str[chem_str['drug_name'].isin(two_sides_drug2list)]

In [57]:
chem_drug1_twosides.info()

NameError: name 'chem_drug1_twosides' is not defined

l=[test1,test2]

DF=pd.concat(l,keys=range(len(l))).groupby(level=1)

maxdf=DF.max()

np.maximum(x1, x2[, out])